In [3]:
import torch
import numpy as np
import scipy.sparse as sp
from numpy.linalg import inv
import pickle

In [4]:
idx_features_labels = np.genfromtxt("./node", dtype=np.dtype(str))
features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)


In [31]:
features.shape,type(features),features.data

((2708, 1433),
 scipy.sparse.csr.csr_matrix,
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))

In [23]:
features.todense(),   type(features.todense()),   features.todense().shape

(matrix([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 numpy.matrix,
 (2708, 1433))

In [10]:
idx_features_labels

array([['31336', '0', '0', ..., '0', '0', 'Neural_Networks'],
       ['1061127', '0', '0', ..., '0', '0', 'Rule_Learning'],
       ['1106406', '0', '0', ..., '0', '0', 'Reinforcement_Learning'],
       ...,
       ['1128978', '0', '0', ..., '0', '0', 'Genetic_Algorithms'],
       ['117328', '0', '0', ..., '0', '0', 'Case_Based'],
       ['24043', '0', '0', ..., '0', '0', 'Neural_Networks']],
      dtype='<U22')

In [9]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [11]:
one_hot_labels = encode_onehot(idx_features_labels[:, -1])


In [13]:
one_hot_labels,one_hot_labels.shape

(array([[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int32),
 (2708, 7))

In [16]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx,idx.shape

(array([  31336, 1061127, 1106406, ..., 1128978,  117328,   24043],
       dtype=int32),
 (2708,))

In [17]:
# build graph
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}#节点：idx
index_id_map = {i: j for i, j in enumerate(idx)}#idx：节点

In [18]:
edges_unordered = np.genfromtxt("./link",dtype=np.int32)
edges_unordered,edges_unordered.shape#一共5259条边

(array([[     35,    1033],
        [     35,  103482],
        [     35,  103515],
        ...,
        [ 853118, 1140289],
        [ 853155,  853118],
        [ 954315, 1155073]], dtype=int32),
 (5429, 2))

In [25]:
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),dtype=np.int32).reshape(edges_unordered.shape)##把节点变换成节点的idx
edges,type(edges),edges.shape

(array([[ 163,  402],
        [ 163,  659],
        [ 163, 1696],
        ...,
        [1887, 2258],
        [1902, 1887],
        [ 837, 1686]], dtype=int32),
 numpy.ndarray,
 (5429, 2))

In [27]:
idx_map[35],idx_map[1033]

(163, 402)

稀疏矩阵的定义（COO、CSC、CSR）
https://www.jianshu.com/p/9671c568096d

In [32]:
(np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1]))# 邻接矩阵里有5429个1，每个1的坐标就是边表的两个数

(array([1., 1., 1., ..., 1., 1., 1.]),
 (array([ 163,  163,  163, ..., 1887, 1902,  837], dtype=int32),
  array([ 402,  659, 1696, ..., 2258, 1887, 1686], dtype=int32)))

In [29]:
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(one_hot_labels.shape[0], one_hot_labels.shape[0]),
                    dtype=np.float32)
adj,type(adj),adj.shape

(<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
 	with 5429 stored elements in COOrdinate format>,
 scipy.sparse.coo.coo_matrix,
 (2708, 2708))

In [37]:
adj.T.multiply(adj.T > adj)

AttributeError: 'numpy.ndarray' object has no attribute 'multiply'

In [ ]:





adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
eigen_adj = None
if self.compute_s:#计算论文中说的图亲密度矩阵
    eigen_adj = self.c * inv((sp.eye(adj.shape[0]) - (1 - self.c) * self.adj_normalize(adj)).toarray())

norm_adj = self.adj_normalize(adj + sp.eye(adj.shape[0]))